In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!unzip /kaggle/input/facial-keypoints-detection/training.zip
!unzip /kaggle/input/facial-keypoints-detection/test.zip

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("training.csv")
test = pd.read_csv('test.csv')

In [ ]:
df.head(1)

In [ ]:
test.head()

In [ ]:
df = df.dropna()

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(np.array(df.Image[45].split(),dtype = int).reshape(96,96),cmap = 'gray')
x = np.array(df.iloc[45,:-1])[::2]
y = np.array(df.iloc[45,:-1])[1::2]
plt.scatter(x,y,c = 'r',s=3)
plt.show()

In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df,transform = None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,index):
        img = torch.tensor(np.array(df.iloc[index,:].Image.split(),dtype = "float32") / 255).reshape(1,96,96).float()
        label = torch.tensor(df.iloc[index,:-1]).float()
        if self.transform is not None:
            img = transform(img)
            return img,label
        return img,label

In [ ]:
from torchvision import transforms

transform = preprocess = transforms.Compose([
    transforms.Normalize(mean=[0.485], std=[0.229]),
])

ds = CustomDataset(df, transform = transform)

In [ ]:
from torch.utils.data import DataLoader,random_split

In [ ]:
train_ds,valid_ds,test_ds = random_split(ds, [1600, len(ds)-1616,16])

In [ ]:
train_dl = DataLoader(train_ds,batch_size = 64,shuffle = True, num_workers = 3, pin_memory= True)
valid_dl = DataLoader(valid_ds,batch_size = 128)
test_dl = DataLoader(test_ds,batch_size = 128)

In [ ]:
from torchvision.models import resnet50 

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = nn.Sequential(
    nn.Conv2d(1,3,3,padding =1),
    resnet50(),
    nn.Linear(1000,100),
    nn.Linear(100,30)
).to(device)

In [ ]:
loss_fn = torch.nn.MSELoss()
loss_fn2 = torch.nn.L1Loss()

In [ ]:
import numpy as np
import torch

class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
def fit(epochs, lr, model, train_dl, valid_dl, opt = torch.optim.Adam):
    
    optimizer = opt(model.parameters(), lr)
    early_stopping = EarlyStopping(patience=20, verbose=True)
    for epoch in range(epochs):
        print(f'{epoch +1} / {epochs}')
        print()
        loss_list_train = []
        loss_list_train2 = []
        loss_list_test = []
        loss_list_test2 = []
        model.train()
        for image, label in train_dl:
            image, label = image.to(device), label.to(device)
            out = model(image)
            loss = loss_fn(out, label)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            with torch.no_grad():                        
                loss2 = loss_fn2(out,label)
            loss_list_train.append(loss.detach().item())
            loss_list_train2.append(loss2.detach().item())

        model.eval()
        for image, label in valid_dl:
            image, label = image.to(device), label.to(device)
            out = model(image)
            loss = loss_fn(out, label)
            loss2 = loss_fn2(out,label)
            loss_list_test.append(loss.detach().item())
            loss_list_test2.append(loss2.detach().item())
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
            

        print("Train_loss",round(sum(loss_list_train) / len(loss_list_train), 4))
        print("Train_loss",round(sum(loss_list_train2) / len(loss_list_train2), 4))
        print("Test_loss", round(sum(loss_list_test) / len(loss_list_test), 4))
        print("Test_loss", round(sum(loss_list_test2) / len(loss_list_test2), 4))


In [ ]:
fit(75,3e-4,model,train_dl,valid_dl)

In [ ]:
for im,lab in test_dl:
    im,lab = im.to(device),lab.to(device)
    out = model(im).detach()
    

In [ ]:
len(out)

In [ ]:
def sep_xy(li):
    x = li[::2]
    y = li[1::2]
    return x,y

In [ ]:
test_ds[0]

In [ ]:
plt.figure(figsize= (15,30))
for idx,lab in enumerate(out):
    plt.subplot(4,4,idx+1)
    plt.imshow(test_ds[idx][0].reshape(96,96), cmap = 'gray')
    x,y = sep_xy(lab.cpu())
    plt.scatter(x,y,c = 'r',s=5)